In [1]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append("..")
import source_code as src

/opt/homebrew/anaconda3/envs/ndvi/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_files(dir_ = '../data/MOD13Q1_061'):
    return [
        os.path.join(dir_, f) for f in os.listdir(dir_) if f.endswith('hdf')
    ]

In [3]:
files = get_files()

In [4]:
files

['../data/MOD13Q1_061/MOD13Q1.A2022353.h13v12.061.2023006034232.hdf']

In [5]:
file = files[0]
file

'../data/MOD13Q1_061/MOD13Q1.A2022353.h13v12.061.2023006034232.hdf'

In [6]:
data, metadata = src.extract_hdf(file, data_field = '250m 16 days NDVI')

In [9]:
data

array([[38910000., 42680000., 42680000., ...,       nan,       nan,
              nan],
       [37750000., 37210000., 39550000., ...,       nan,       nan,
              nan],
       [39440000., 38380000., 41420000., ...,       nan,       nan,
              nan],
       ...,
       [20190000., 20910000., 21180000., ...,       nan,       nan,
              nan],
       [22140000., 22360000., 21120000., ...,       nan,       nan,
              nan],
       [21080000., 22620000., 22620000., ...,       nan,       nan,
              nan]], shape=(4800, 4800))

In [7]:
file_bounding_box = src.convert_to_epsg4326(metadata)

src.bounding_box_to_polygon(file_bounding_box).wkt

'POLYGON ((-65.27036445731774 -39.99999999641088, -46.18802152977182 -39.99999999641088, -46.18802152977182 -29.99999999730591, -65.27036445731774 -29.99999999730591, -65.27036445731774 -39.99999999641088))'

In [8]:
metadata

{'SwathStructure': {},
 'GridStructure': {'GRID_1': {'GridName': 'MODIS_Grid_16DAY_250m_500m_VI',
   'XDim': 4800,
   'YDim': 4800,
   'UpperLeftPointMtrs': (-5559752.598333, -3335851.559),
   'LowerRightMtrs': (-4447802.078667, -4447802.078667),
   'Projection': 'GCTP_SNSOID',
   'ProjParams': (6371007.181,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0),
   'SphereCode': '-1',
   'Dimension': {},
   'DataField': {'DataField_1': {'DataFieldName': '250m 16 days NDVI',
     'DataType': 'DFNT_INT16',
     'DimList': ['YDim', 'XDim']},
    'DataField_2': {'DataFieldName': '250m 16 days EVI',
     'DataType': 'DFNT_INT16',
     'DimList': ['YDim', 'XDim']},
    'DataField_3': {'DataFieldName': '250m 16 days VI Quality',
     'DataType': 'DFNT_UINT16',
     'DimList': ['YDim', 'XDim']},
    'DataField_4': {'DataFieldName': '250m 16 days red reflectance',
     'DataType': 'DFNT_INT16',
     'DimList': ['YDim', 'XDim']},
    'DataFie

In [ ]:
def convert_sinu_to_dataframe(data3D):
    
    data = data3D.get().astype(float)

    # Read attributes.
    attrs = data3D.attributes(full=1)
    lna=attrs["long_name"]
    long_name = lna[0]
    vra=attrs["valid_range"]
    valid_range = vra[0]
    fva=attrs["_FillValue"]
    _FillValue = fva[0]
    sfa=attrs["scale_factor"]
    scale_factor = sfa[0]        
    ua=attrs["units"]
    units = ua[0]
    aoa=attrs["add_offset"]
    add_offset = aoa[0]

    # Apply the attributes to the data.
    invalid = np.logical_or(data < valid_range[0], data > valid_range[1])
    invalid = np.logical_or(invalid, data == _FillValue)
    data[invalid] = np.nan
    data = (data - add_offset) * scale_factor
    data = np.ma.masked_array(data, np.isnan(data))

    # Construct the grid.  The needed information is in a global attribute
    # called 'StructMetadata.0'.  Use regular expressions to tease out the
    # extents of the grid.
    fattrs = hdf.attributes(full=1)
    ga = fattrs["StructMetadata.0"]
    gridmeta = ga[0]
    ul_regex = re.compile(r'''UpperLeftPointMtrs=\(
                              (?P<upper_left_x>[+-]?\d+\.\d+)
                              ,
                              (?P<upper_left_y>[+-]?\d+\.\d+)
                              \)''', re.VERBOSE)

    match = ul_regex.search(gridmeta)
    x0 = np.float64(match.group('upper_left_x'))
    y0 = np.float64(match.group('upper_left_y'))

    lr_regex = re.compile(r'''LowerRightMtrs=\(
                              (?P<lower_right_x>[+-]?\d+\.\d+)
                              ,
                              (?P<lower_right_y>[+-]?\d+\.\d+)
                              \)''', re.VERBOSE)
    match = lr_regex.search(gridmeta)
    x1 = np.float64(match.group('lower_right_x'))
    y1 = np.float64(match.group('lower_right_y'))
    nx, ny = data.shape
    
    # Generate the sinusoidal grid
    x = np.linspace(x0, x1, nx)
    y = np.linspace(y0, y1, ny)
    xv, yv = np.meshgrid(x, y)

    # Define projection transformation
    sinu = pyproj.Proj("+proj=sinu +R=6371007.181 +nadgrids=@null +wktext")
    wgs84 = pyproj.Proj("+init=EPSG:4326")

    # Convert all points to WGS84
    lon, lat = pyproj.transform(sinu, wgs84, xv, yv)

    data_list = []
    
    # Compute pixel size
    pixel_width = (x1 - x0) / nx
    pixel_height = (y1 - y0) / ny

    for i in range(ny - 1):
        for j in range(nx - 1):
            # Define pixel corners in sinusoidal projection
            min_x, min_y = xv[i, j], yv[i, j]
            max_x, max_y = min_x + pixel_width, min_y + pixel_height

            # Convert pixel corners to WGS84
            ll = pyproj.transform(sinu, wgs84, min_x, min_y)  # Lower-left
            lr = pyproj.transform(sinu, wgs84, max_x, min_y)  # Lower-right
            ur = pyproj.transform(sinu, wgs84, max_x, max_y)  # Upper-right
            ul = pyproj.transform(sinu, wgs84, min_x, max_y)  # Upper-left

            # Create WKT polygon
            polygon = Polygon([ll, lr, ur, ul, ll])
            data_list.append((data[i, j], polygon.wkt))

    # Create DataFrame
    df = pd.DataFrame(data_list, columns=["value", "wkt"])
    
    return df